In [1]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn

from util.ZigZag import ZigZag
from util.BuildData import BuildData4
from util.CalcReturns import CalcReturns

from sklearn import svm, preprocessing

In [2]:
def recurrent_neural_network_model(x):

    layer = {'w' : tf.Variable(tf.random_normal([num_hidden, n_classes])),
             'b' : tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.unstack(x, timesteps, 1)    
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)    
    outs, state = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)        
    y = tf.add(tf.matmul(outs[-1], layer['w']), layer['b'], name='nn')
    
    return y

In [3]:
def train_and_save_nn(x,y):
    nn = recurrent_neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = nn, labels = y))
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(cost)   
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.0006).minimize(cost)   
    
    correct = tf.equal(tf.argmax(nn,1), tf.argmax(y_train, 1), name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'), name = 'accuracy')
    
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.initialize_all_variables())

        epoch_loss = 0
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for i in range(int((len(D) - t_count)/batch_size)):
                
                batch_x = D[i*batch_size : (i+1)*batch_size]
                batch_y = L[i*batch_size : (i+1)*batch_size]
               
                batch_x = batch_x.reshape((batch_size, timesteps, num_input))
                
                _, c = session.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
                
            if epoch % 100 == 0:
                print('Epoch', epoch, 'completed out of', hm_epochs, 'loss', epoch_loss)
        
        
        
                
        saver.save(session, 'data/fxnn.ckpt')
        
        print('Last epoch loss: ', epoch_loss)
        return nn, correct, accuracy

In [4]:
def test_nn(x_test, y_test):
    with tf.Session() as session:
        saver = tf.train.import_meta_graph('data/fxnn.ckpt.meta')
        saver.restore(session, tf.train.latest_checkpoint('data/'))
        
        graph    = tf.get_default_graph()
        correct  = graph.get_tensor_by_name('correct:0')
        accuracy = graph.get_tensor_by_name('accuracy:0')
        
        x = graph.get_tensor_by_name('x:0')
        y = graph.get_tensor_by_name('y:0')

        x_test = x_test.reshape((t_count, timesteps, num_input))
        
        r = tf.cast(correct, 'float').eval({x:x_test, y:y_test})       

        #for i in range(len(r)):
        #    print '%s %s' % (r[i], y_test[i])        

        test_buy  = 0.0
        test_sell = 0.0

        correct_buy  = 0.0
        correct_sell = 0.0

        incorrect_buy  = 0.0
        incorrect_sell = 0.0

        for i in range(len(r)):
            if y_test[i][1] == 1:
                test_buy += 1.0;
            elif y_test[i][0] == 1:
                test_sell += 1.0;

            if r[i] == 1:
                if y_test[i][1] == 1:
                    correct_buy += 1.0;
                elif y_test[i][0] == 1:
                    correct_sell += 1.0;
            else:
                if y_test[i][1] == 1:
                    incorrect_buy += 1.0;
                elif y_test[i][0] == 1:
                    incorrect_sell += 1.0;

        print('Accuracy:',accuracy.eval({x:x_test, y:y_test}))

        print( "Test buy:  ", test_buy  )
        print( "Test sell: ", test_sell )

        print( "Correct buy:  ", correct_buy,  "Incorrect buy:  ", incorrect_buy,  "Accuracy: ", (correct_buy/test_buy)*100 )
        print( "Correct sell: ", correct_sell, "Incorrect sell: ", incorrect_sell, "Accuracy: ", (correct_sell/test_sell)*100 )

In [5]:
n_classes  = 2

step = 61

timesteps = 10
num_input = 6

#step - 1 == num_input * timesteps

hm_epochs  = 1000
num_hidden = 16
batch_size = 100


##########################################################################

df = pd.read_csv('EURUSD1.csv', header=0)
train = np.array(df.Close)

Z = ZigZag(train, 0.004)
R = CalcReturns(train)
S = BuildData4(Z, R, step)

t_count = 1024
dims = S["data"].shape[1]

D = S["data"]
L = []

for i in S["label"]:
    if i == 1:
        L.append([0, 1])
    elif i == -1:
        L.append([1, 0])

L = np.array(L)
        
#print D.shape
#print L.shape

x_train = tf.placeholder('float', [None, timesteps, num_input], name='x')
y_train = tf.placeholder('float', [None, n_classes], name='y')

train_and_save_nn(x_train, y_train)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Epoch', 0, 'completed out of', 1000, 'loss', 75.849908918142319)
('Epoch', 100, 'completed out of', 1000, 'loss', 73.281341761350632)
('Epoch', 200, 'completed out of', 1000, 'loss', 73.208497136831284)
('Epoch', 300, 'completed out of', 1000, 'loss', 73.172708928585052)
('Epoch', 400, 'completed out of', 1000, 'loss', 73.153834134340286)
('Epoch', 500, 'completed out of', 1000, 'loss', 73.140081465244293)
('Epoch', 600, 'completed out of', 1000, 'loss', 73.130677491426468)
('Epoch', 700, 'completed out of', 1000, 'loss', 73.123972982168198)
('Epoch', 800, 'completed out of', 1000, 'loss', 73.118910074234009)
('Epoch', 900, 'completed out of', 1000, 'loss', 73.114973664283752)
('Last epoch loss: ', 73.111904412508011)


(<tf.Tensor 'nn:0' shape=(?, 2) dtype=float32>,
 <tf.Tensor 'correct:0' shape=(?,) dtype=bool>,
 <tf.Tensor 'accuracy:0' shape=() dtype=float32>)

In [6]:
x_test = D[len(D)-t_count:]
y_test = L[len(L)-t_count:]

test_nn(x_test, y_test)

('Accuracy:', 0.52539062)
('Test buy:  ', 486.0)
('Test sell: ', 538.0)
('Correct buy:  ', 0.0, 'Incorrect buy:  ', 486.0, 'Accuracy: ', 0.0)
('Correct sell: ', 538.0, 'Incorrect sell: ', 0.0, 'Accuracy: ', 100.0)
